Vamos a cargar las librerias y datos necesarios:

In [63]:
import pandas as pd
import tensorflow as tf
import numpy as np


uefa_ranking = pd.read_csv('data/uefa_ranking.csv')
goles = pd.read_csv('data/equipos-goles.csv')
ultimas_victorias = pd.read_csv('data/victorias.csv')
champions = pd.read_csv('data/champions_wins.csv')


# Crear un diccionario para mapear nombres de equipos a números únicos
team_mapping = {team: idx + 1 for idx, team in enumerate(uefa_ranking['Club'])}


#Funcion puntaje para la prediccion de partidos
def score_prediction(team, tarjetaA=0, tarjetaR=0, foulC=0, foulS=0):
    uefa_score = uefa_ranking.loc[uefa_ranking['Club'] == team, 'Total Points'].values[0]
    ngoles = goles.loc[goles['Equipo'] == team, 'Goles'].values[0] if not goles.loc[goles['Equipo'] == team, 'Goles'].empty else 0
    victorias = ultimas_victorias.loc[ultimas_victorias['Equipo'] == team, 'Victorias'].values[0] if not ultimas_victorias.loc[ultimas_victorias['Equipo'] == team, 'Victorias'].empty else 0
    nchampions = champions.loc[champions['Equipo'] == team, 'Champions'].values[0] if not champions.loc[champions['Equipo'] == team, 'Champions'].empty else 0
    return uefa_score + ngoles + victorias + foulS - tarjetaA - tarjetaR*2 - foulC + nchampions

<h4>Cuartos, primer partido: Atlético Madrid vs Borussia Dortmund

In [64]:
equipo1 = "Atlético Madrid"
equipo2 = "Borussia Dortmund"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = score_prediction(equipo1, 26, 1, 115, 86)
total_points_equipo2 = score_prediction(equipo2, 18, 0, 116, 101)

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


El número de la columna 'Total Points' correspondiente al equipo Atlético Madrid es: 234.0
El número de la columna 'Total Points' correspondiente al equipo Borussia Dortmund es: 243.0


In [65]:
def predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping):
    # Cargar el modelo
    model = tf.keras.models.load_model("model2.h5")

    # Mapear los nombres de los equipos a los números únicos usando team_mapping
    equipo1_map = team_mapping[equipo1]
    equipo2_map = team_mapping[equipo2]

    # Definir la información del partido
    round_number = 9  # Ronda correspondiente

    # Crear una matriz de características para la predicción
    prediction_features = np.array([[round_number, equipo1_map, equipo2_map, total_points_equipo1, total_points_equipo2]])

    # Hacer la predicción utilizando el modelo cargado
    prediction1 = model.predict(prediction_features)

    # Imprimir la predicción
    print(f"Probabilidad de que gane {equipo1} en la ronda 9 es:")
    print(prediction1)

    # Definir la información del partido para la siguiente ronda
    round_number = 10  # Ronda correspondiente

    # Intercambiar los equipos para la siguiente predicción
    equipo1_map, equipo2_map = equipo2_map, equipo1_map
    total_points_equipo1, total_points_equipo2 = total_points_equipo2, total_points_equipo1

    # Crear una matriz de características para la predicción de la siguiente ronda
    prediction_features = np.array([[round_number, equipo1_map, equipo2_map, total_points_equipo1, total_points_equipo2]])

    # Hacer la predicción utilizando el modelo cargado
    prediction2 = model.predict(prediction_features)

    # Imprimir la predicción
    print(f"Probabilidad de que gane {equipo2} en la ronda 10 es:")
    print(prediction2)

    # Calcular el porcentaje de predicción de cada equipo
    percentage_prediction1 = prediction1[0][0]
    percentage_prediction2 = prediction2[0][0]

    # Calcular la probabilidad acumulada de cada equipo
    cumulative_probability1 = (percentage_prediction1 + (1 - percentage_prediction2))/2
    cumulative_probability2 = (percentage_prediction2 + (1 - percentage_prediction1))/2

    # Determinar el ganador
    if cumulative_probability1 > cumulative_probability2:
        winner = equipo1
    else:
        winner = equipo2

    # Retornar el ganador
    return winner

# Predecir el ganador del partido entre "Atlético Madrid" y "Borussia Dortmund"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Probabilidad de que gane Atlético Madrid en la ronda 9 es:
[[0.21176827]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Probabilidad de que gane Borussia Dortmund en la ronda 10 es:
[[0.3256074]]
El ganador del partido entre Atlético Madrid y Borussia Dortmund es: Borussia Dortmund


<h4>Cuartos, segundo partido: Paris Saint-Germain vs FC Barcelona

In [66]:
equipo1 = "Paris Saint-Germain"
equipo2 = "FC Barcelona"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = score_prediction(equipo1, 28, 0, 126, 120)
total_points_equipo2 = score_prediction(equipo2, 31, 3, 104, 117)

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


El número de la columna 'Total Points' correspondiente al equipo Paris Saint-Germain es: 343.0
El número de la columna 'Total Points' correspondiente al equipo FC Barcelona es: 377.0


In [67]:
# Predecir el ganador del partido entre "Paris Saint-Germain" y "FC Barcelona"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Probabilidad de que gane Paris Saint-Germain en la ronda 9 es:
[[0.21175884]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Probabilidad de que gane FC Barcelona en la ronda 10 es:
[[0.4451071]]
El ganador del partido entre Paris Saint-Germain y FC Barcelona es: FC Barcelona


<h4>Cuartos, tercer partido: Arsenal vs Bayern München

In [68]:
equipo1 = "Arsenal"
equipo2 = "Bayern München"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = score_prediction(equipo1, 14, 0, 129, 109)
total_points_equipo2 = score_prediction(equipo2, 15, 1, 112, 109)

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


El número de la columna 'Total Points' correspondiente al equipo Arsenal es: 137.0
El número de la columna 'Total Points' correspondiente al equipo Bayern München es: 508.0


In [69]:
# Predecir el ganador del partido entre "Arsenal" y "Bayern München"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Probabilidad de que gane Arsenal en la ronda 9 es:
[[0.12390766]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Probabilidad de que gane Bayern München en la ronda 10 es:
[[0.77128613]]
El ganador del partido entre Arsenal y Bayern München es: Bayern München


<h4>Cuartos, cuarto partido: Real Madrid vs Manchester City

In [70]:
equipo1 = "Real Madrid"
equipo2 = "Manchester City"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = score_prediction(equipo1, 20, 0, 110, 111)
total_points_equipo2 = score_prediction(equipo2, 10, 0, 74, 125)

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


El número de la columna 'Total Points' correspondiente al equipo Real Madrid es: 547.0
El número de la columna 'Total Points' correspondiente al equipo Manchester City es: 501.0


In [71]:
# Predecir el ganador del partido entre "Real Madrid" y "Manchester City"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Probabilidad de que gane Real Madrid en la ronda 9 es:
[[0.4451071]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Probabilidad de que gane Manchester City en la ronda 10 es:
[[0.21175884]]
El ganador del partido entre Real Madrid y Manchester City es: Real Madrid


<h3>Semis, primer partido: Borussia Dortmund vs FC Barcelona (predicción inicial)

In [72]:
equipo1 = "Borussia Dortmund"
equipo2 = "FC Barcelona"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = score_prediction(equipo2, 18, 0, 116, 101)
total_points_equipo2 = score_prediction(equipo2, 31, 3, 104, 117)

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


El número de la columna 'Total Points' correspondiente al equipo Borussia Dortmund es: 368.0
El número de la columna 'Total Points' correspondiente al equipo FC Barcelona es: 377.0


In [73]:
# Predecir el ganador del partido entre "Borussia Dortmund" y "FC Barcelona"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Probabilidad de que gane Borussia Dortmund en la ronda 9 es:
[[0.21176271]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Probabilidad de que gane FC Barcelona en la ronda 10 es:
[[0.43403956]]
El ganador del partido entre Borussia Dortmund y FC Barcelona es: FC Barcelona


<h3>Semis, primer partido: Borussia Dortmund vs Paris Saint-Germain

In [74]:
equipo1 = "Borussia Dortmund"
equipo2 = "Paris Saint-Germain"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = score_prediction(equipo1, 18, 0, 116, 101)
total_points_equipo2 = score_prediction(equipo2, 28, 0, 126, 120)

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


El número de la columna 'Total Points' correspondiente al equipo Borussia Dortmund es: 243.0
El número de la columna 'Total Points' correspondiente al equipo Paris Saint-Germain es: 343.0


In [75]:
# Predecir el ganador del partido entre "Borussia Dortmund" y "Paris Saint-Germain"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Probabilidad de que gane Borussia Dortmund en la ronda 9 es:
[[0.13642538]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Probabilidad de que gane Paris Saint-Germain en la ronda 10 es:
[[0.82387644]]
El ganador del partido entre Borussia Dortmund y Paris Saint-Germain es: Paris Saint-Germain


<h3>Semis, segundo partido: Bayern München vs Real Madrid (predicción inicial)

In [76]:
equipo1 = "Bayern München"
equipo2 = "Real Madrid"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = score_prediction(equipo1, 15, 1, 112, 109)
total_points_equipo2 = score_prediction(equipo2, 20, 0, 110, 111)

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)

El número de la columna 'Total Points' correspondiente al equipo Bayern München es: 508.0
El número de la columna 'Total Points' correspondiente al equipo Real Madrid es: 547.0


In [77]:
# Predecir el ganador del partido entre "Bayern München" y "Real Madrid"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Probabilidad de que gane Bayern München en la ronda 9 es:
[[0.21175884]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Probabilidad de que gane Real Madrid en la ronda 10 es:
[[0.4451071]]
El ganador del partido entre Bayern München y Real Madrid es: Real Madrid


<h2>Final: FC Barcelona vs Real Madrid (predicción inicial)

In [78]:
equipo1 = "FC Barcelona"
equipo2 = "Real Madrid"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = score_prediction(equipo1, 31, 3, 104, 117)
total_points_equipo2 = score_prediction(equipo2, 20, 0, 110, 111)

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)

El número de la columna 'Total Points' correspondiente al equipo FC Barcelona es: 377.0
El número de la columna 'Total Points' correspondiente al equipo Real Madrid es: 547.0


In [79]:
# Predecir el ganador del partido entre "FC Barcelona" y "Real Madrid"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Probabilidad de que gane FC Barcelona en la ronda 9 es:
[[0.13631794]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Probabilidad de que gane Real Madrid en la ronda 10 es:
[[0.8250693]]
El ganador del partido entre FC Barcelona y Real Madrid es: Real Madrid


<h1>Por lo tanto, el ganador de la Champions es: REAL MADRID!

<h2>Final: PSG vs Real Madrid (2ª predicción inicial)

In [80]:
equipo1 = "Paris Saint-Germain"
equipo2 = "Real Madrid"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = score_prediction(equipo1, 28, 0, 126, 120)
total_points_equipo2 = score_prediction(equipo2, 20, 0, 110, 111)

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


El número de la columna 'Total Points' correspondiente al equipo Paris Saint-Germain es: 343.0
El número de la columna 'Total Points' correspondiente al equipo Real Madrid es: 547.0


In [81]:
# Predecir el ganador del partido entre "Paris Saint-Germain" y "Real Madrid"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Probabilidad de que gane Paris Saint-Germain en la ronda 9 es:
[[0.13631792]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Probabilidad de que gane Real Madrid en la ronda 10 es:
[[0.75811684]]
El ganador del partido entre Paris Saint-Germain y Real Madrid es: Real Madrid


<h1>Por lo tanto, el ganador de la Champions es: REAL MADRID!